importing json module and reading data as well as checking type, format of data

In [1]:
import json
from time import time
import random
tag_count={}
#defining dictionary for tag count
now=time()
f = open('penn-data.json')
data = json.load(f)
#print(data)
random.shuffle(data)
#adding each tag top dictionary
for point in data:
    tags=point[1]
    for tag in tags:
        if tag not in tag_count:
            tag_count[tag]=1
        else:
            tag_count[tag]+=1
freq=0
most_used_tag=''
total=0
#getting most used tag for case of missing word
for tag,count in tag_count.items():
    total+=count
    if count>freq:
        freq=count
        most_used_tag=tag
        
print('Total number of words is:',total )        
print('Most used tag is: ',most_used_tag)
print(f'It is used for {freq} times ')
print("total number of tags in dataset is: ",len(tag_count))

Total number of words is: 81147
Most used tag is:  NN
It is used for 12721 times 
total number of tags in dataset is:  41


splitting data into train and test in ration of 80:20.


In [2]:
train_nos=round(0.8*len(data))#total number of training examples  
print('Total number of examples: ',len(data))
print('No. of training examples: ',train_nos)
test_nos=len(data)-train_nos#total number of test examples
print('No. of testing examples: ',test_nos)

Total number of examples:  3914
No. of training examples:  3131
No. of testing examples:  783


Preprocessing
1. Removing special characters such as comms, inverted commas, full-stops.
2. Lowering each character so that same word cannot oocupy different serial numbers which are being assigned next step.
3. Adding start and end tag at begining and ending of each sequence and corrosponding tags.
Above pre processed sequences and tag lists are written in seq.txt and tag.txt.

Tokenization
Preparing dictionary of words in test data set for better access. Assigning unique serial number to each word.(word_dict)
Similar dictionary for tags is prepared in above cells.(tag_dict)

Tokenization will also give total number of unique words, tags which can be used in later stages if required.

In [3]:
word_dict={'<start>':0}
tag_dict={'<start>':0}
word_count=1
tag_count=1
seq_file = open("seq.txt","w")
tag_file=open("tag.txt","w")
for tag_seq in data:
    seq=tag_seq[0].split(' ')
    tag=tag_seq[1]
    for i in range(len(seq)):
        seq[i]=seq[i].lower()
        if "'" in seq[i]:
            seq[i]=seq[i].replace("'",'',100)
        if '.' in seq[i]:
            seq[i]=seq[i].replace('.','',100)
        if ',' in seq[i]:
            seq[i]=seq[i].replace(',','',100)
        if ',' in seq[i]:
            seq[i]=seq[i].replace('"','',100)
        if '``' in seq[i]:
            seq[i]=seq[i].replace('``','',100)
        if '?' in seq[i]:
            seq[i]=seq[i].replace('``','',100)
        if seq[i]=='&':
            seq[i]='and'
            
            
    seq=['<start>']+seq+['<end>']
    out_seq=' '
    out_tag=' '
    out_seq=out_seq.join(seq)
    
    tag=['<start>']+tag+['<end>']
    out_tag=out_tag.join(tag)
    seq_file.write(out_seq)
    seq_file.write('\n')
    tag_file.write(out_tag)
    tag_file.write('\n')


seq_file.close()
tag_file.close()

Preparing test dataset. 
Now we have tags and sequences in required format. 
That data is splitted in below step. It is being stored seq_train.txt amnd tag_train.txt files.
during writing tags and sequences they are being written using tokens in word_dict and tag_dict so that we can easily use them in later stages. 

In [4]:
tag_file=open("tag.txt","r")
seq_file = open("seq.txt","r")
train_seq=open("seq_train.txt","w")
train_tag=open("tag_train.txt","w")
seq_file.seek(0)
tag_file.seek(0)
word_dict={'<start>':0}
tag_dict={'<start>':0}
word_count=1
tag_count=1
for i in range(train_nos):
    temp=seq_file.readline().split(' ')
    out=' '
    for word in temp:
        if word not in word_dict:
            word_dict[word]=word_count
            word_count+=1
    x=[str(word_dict[word]) for word in temp]
    out=out.join(x)
    train_seq.write(out)
    train_seq.write('\n')
    temp=tag_file.readline().split(' ')
    out=' '
    for tag in temp:
        if tag not in tag_dict:
            tag_dict[tag]=tag_count
            tag_count+=1
    x=[str(tag_dict[tag]) for tag in temp]
    out=out.join(x)
    train_tag.write(out)
    train_tag.write('\n')
test_tag=open('test_tag.txt','w')
test_seq=open('test_seq.txt','w')
for j in range(train_nos,len(data)):
    temp_seq=seq_file.readline()
    temp_tag=tag_file.readline()
    test_tag.write(temp_tag)
    #test_tag.write('\n')
    test_seq.write(temp_seq)
    #test_seq.write('\n')
train_tag.close()
train_seq.close()
tag_file.close()
seq_file.close()
test_tag.close()
test_seq.close()

algorithm may encounter a case where some specific tag is present in test dataset but not in train dataset(foreign tag). For that case all tags must be considered. Though transisition and initial probabilities will be zero for corrosponding tag but they must be there to ensure uninterupted working of algorithm. 

For a case of foereign word we are considering most used for that word. 

In [5]:
tag_file=open('tag.txt','r')
for i in range(len(data)):
    temp=tag_file.readline().split(' ')
    for tag in temp:
            if tag not in tag_dict:
                tag_dict[tag]=tag_count
                tag_count+=1
tag_file.close()

preparing emission probability matrix and transisition probability matrix. 
Dimensions for emission probability matrix: word_count x tag_count
dimension for transisition probability matrix: tag_count x tag_count

In [6]:
import numpy as np
emit=np.zeros([word_count,tag_count])
trans=np.zeros([tag_count,tag_count])

calculating both matrices.

In [7]:
def matrix_calculator(emit,trans,train_nos):
    train_seq=open('seq_train.txt','r')
    train_tag=open('tag_train.txt','r')
    train_seq.seek(0)
    train_tag.seek(0)
    for i in range(train_nos):
        temp_seq=train_seq.readline()
        temp_tag=train_tag.readline()
        input_seq=[int(w) for w in temp_seq.split(' ')]
        #parasing each sequence in train sequence 
        input_tag=[int(t) for t in temp_tag.split(' ')]
        #arsing each tag in train tags
        for i in range(len(input_seq)):
            emit[input_seq[i]][input_tag[i]]+=1
            #incrementing corrosponding count of emission probability matrix
        for j in range(len(input_tag)-1):
            trans[input_tag[j]][input_tag[j+1]]+=1
            #incrementing corrosponding count of transmission probability matrix

    train_seq.close()
    train_tag.close()
    emit=emit/emit.sum(axis=0, keepdims=True)
    #dividing each element by sum of elements in corrosdponding column
    trans=trans/trans.sum(axis=1, keepdims=True)
    #dividing each element by sum of elements in corrosdponding row
    return emit,trans
train_nos=round(0.8*len(data))
emit=np.zeros([word_count,tag_count])
trans=np.zeros([tag_count,tag_count])
emit,trans=matrix_calculator(emit,trans,train_nos)

<ipython-input-7-f60bf03b76b6>:24: RuntimeWarning: invalid value encountered in true_divide
  trans=trans/trans.sum(axis=1, keepdims=True)


In [8]:
#verifying matrix 
trans[tag_dict['MD']][tag_dict['DT']]

0.00410958904109589

Now that we have found transition and emission matrix we will proceed toward testing.

In [9]:
#getting data for number of missing words
test_seq=open("test_seq.txt",'r')
test_seq.seek(0)
missing=0
for line in test_seq:
    for word in line.split(' '):
        if word not in word_dict:
            missing+=1
print('Number of missing words is: ',missing)
test_seq.close()

Number of missing words is:  1368


In [10]:
#defing sdtart tag num and end tag num
start_tag_num=word_dict['<start>']
end_tag_num=tag_dict['<end>\n']

For testing we have used OOP approach. 
Each node (node class) is object with attributes:
1. tag_num: tag number for corrosponding time step
2. word: word at that time step
3. emit_prob: emission probability of word and that tag
4. parent: best possible node tag number in previous tag(used in bigram model)
5. grandparent: best possible node tag number based on previous two tags
6. grandparent_prob: transistion probability * emission probability based on previous two nodes
7. grandparent_node: best possible node based on previous two timesteps
in node object we have defined a step for foreign word. In that step if word is foreign word then node with most used tag is having emnission probability one else it will be zero.

each time step (class time_instance) is object with attributes:
1. word=word at at that timestep
2. tag_count=total number of nodes at that particular timesteps
3. nodes=it is list ciontating all node objects at that particular timestep

timeline object
It is basially our whole structure through which our path will move and output is obtained using viterbi algorithm. 
It is having following attributes:
1. seq=Input sequence for which tag sequence is to be determined
2. tag_count=total number of tags in whole dataset 
3. timesteps=timestep object for each word created using above class

In [11]:
class node:
    out=[]
    def __init__(self,tag_num,word,level):
        self.tag_num=tag_num
        self.word=word
        self.emit_prob=None
        self.parent=None
        self.grandparent=None
        self.grandparent_prob=0
        self.grandparent_node=None
        self.level=level
        if self.word not in word_dict:
            if self.tag_num!=tag_dict[most_used_tag]:
                self.emit_prob=0
            else:
                self.emit_prob=1
        else:
            self.emit_prob=emit[word_dict[word]][self.tag_num]
    
    def result(self):
        while self.grandparent:
            #print(self.tag_num)
            node.out.append(self.tag_num)
            return self.grandparent.result()
            
class time_instance:
    def __init__(self,word,level):
        self.word=word
        self.tag_count=tag_count
        self.nodes=[]
        
        for i in range(self.tag_count):
            #appending number of nodes equal to number of tags
            self.nodes.append(node(i,word,level))
        #print(f'timestep created with {len(self.nodes)} nodes')
        '''self.nodes=[node for node in self.nodes1 if node.emit_prob!=0]'''
        

class timeline:
    def __init__(self,seq):
        self.seq=seq
        self.tag_count=tag_count
        self.timesteps=[]
        #appending number of timesteps equal to number of words in input sequence
        for i in range(len(self.seq)):
            self.timesteps.append(time_instance(seq[i],i))
        #print(f'timeline created with {len(self.timesteps)} timesteps')
            
    def parent_generator(self):
        #this method is used if bigram model in cosidered for given sequence
        #for first 2 words
        #print('parent method called')
        t1=self.timesteps[1]
        ##bigram model is considerd for word at index 1
        temp=0
        first_tag=0
        for node in (t1.nodes):
            if node.emit_prob!=0:
                #selecting node with maximum initial probability
                if temp<trans[0][node.tag_num]*node.emit_prob:
                    temp=trans[0][node.tag_num]*node.emit_prob
                    first_tag=node
        x=first_tag.tag_num
        return x
     #grand_parent_generator method is used for finding node with trigram assumption   
    def grand_parent_generator(self):
        #defining rult matrix
        result=[0]
        result.append(self.parent_generator())
        #print('grand parent method called')
        for time in range(2,len(self.timesteps)):
            #print(f'{time}th timeline analysis')
            #t2 is current timestep
            t2=self.timesteps[time]
            #time step before current
            t1=self.timesteps[time-1]
            #t0 is timestep before above timestep
            t0=self.timesteps[time-2]
            for n2 in range(len(t2.nodes)):
                #print(f'{n2}th node')
                #if for particular node emission probability is zero then that node is not considered as final probability will be also
                if t2.nodes[n2].emit_prob!=0:
                    temp_prob=0
                    #tag_c is tag_num of node at current time 
                    tag_c=t2.nodes[n2].tag_num
                    for n1 in range(len(t1.nodes)):
                        #tag_b is tag_num at node at first previous timestep
                        tag_b=t1.nodes[n1].tag_num
                        #t_b_c is transition probability with tag_b
                        t_b_c=trans[tag_b][tag_c]
                        #if t_b_c is zero no path will be considered to reduce calculation
                        if t_b_c!=0:
                            for n0 in range(len(t0.nodes)):
                                #tag_a is tag_num at second previous node
                                tag_a=t0.nodes[n0].tag_num
                                #t_b_c is transmission probability of current node with second previous node
                                t_a_b=trans[tag_a][tag_b]
                                if t_a_b*t2.nodes[n2].emit_prob>0:
                                    #choosing node with max probability
                                    if temp_prob<=t_a_b*t_b_c:
                                        temp_prob=t_a_b*t_b_c
                                        #updating grandparent object
                                        t2.nodes[n2].grandparent=t1.nodes[n1]
                                        #updating that probability at current node
                                        t2.nodes[n2].grandparent_prob=temp_prob*t2.nodes[n2].emit_prob
        #result evaluation    
        #considering path with max probability
        for j in range(len(self.timesteps)):
            last_nodes=self.timesteps[j].nodes
            result_prob=0
            result_node=None
            for i in range(len(last_nodes)):
                if last_nodes[i].emit_prob!=0:
                    if result_prob<=last_nodes[i].grandparent_prob:
                        result_prob=last_nodes[i].grandparent_prob
                        result_node=last_nodes[i]
            result.append(result_node.tag_num)
        x=result
        #returning result
        return x

Now using algorithm, each test input in given and accuracy is calculated

In [12]:
#opening test sequence and corrosponding tag file
test_seq=open('test_seq.txt','r')
test_tag=open('test_tag.txt','r')
test_seq.seek(0)
test_tag.seek(0)
out_tag=[]#all reulting tags
expected_tag=[]#all expected tags
failed_expected_tag=[]
failed_cases_initial=0#if test case fails in algorithm then this variable is incremented
failed_cases_mismatch=0##if test case with dimension then this variable is incremented    
for test_case in range(test_nos):
    #print(test_case)
    a=test_seq.readline().split(' ')
    b=[tag_dict[i] for i in test_tag.readline().split(' ')]
    try:
        test_timeline=timeline(a)
        result=test_timeline.grand_parent_generator()#for each sentense grand parent_generator method is called
        del result[2:4]
        out_tag.append(result)
        expected_tag.append(b)
    except:
        failed_cases_initial+=1
        print(a)
        print(f'failed at {test_case}')
test_seq.close()
test_tag.close()
true=0
false=0
true_sentense=0
false_sentense=0
#calculating accuracy
for i in range(len(out_tag)):
    a=out_tag[i]
    b=expected_tag[i]
    if len(a)==len(b):
        for j in range(len(out_tag[i])):
            if a[j]==b[j]:
                true+=1
            else:
                false+=1
        if a==b:
            true_sentense+=1
        else:
            false_sentense+=1
    else:
        failed_cases_mismatch+=1
acc=true/(true+false)
acc_sentense=true_sentense/(true_sentense+false_sentense)
#printing result
print(f'accuracy is: {acc*100} %')
print(f'Sentense wise accuracy is: {acc_sentense*100}%' )
later=time()
print('Total number of cases checked is: ',test_nos)
print('Number of failed cases is: ', failed_cases_initial)
print('number of failed cases in mismatch are: ',failed_cases_mismatch)
print(f'time required for execution is {later-now} seconds') 



accuracy is: 85.92973061581336 %
Sentense wise accuracy is: 7.2796934865900385%
Total number of cases checked is:  783
Number of failed cases is:  0
number of failed cases in mismatch are:  0
time required for execution is 55.359166622161865 seconds


In [13]:
#getting tag wose accuracy
out_dict={i:0 for i in range(tag_count)}
exp_dict={i:0 for i in range(tag_count)}
for i in range(len(out_tag)):
    for j in range(len(out_tag[i])):
        exp_dict[out_tag[i][j]]+=1
        if out_tag[i][j]==expected_tag[i][j]:
            out_dict[out_tag[i][j]]+=1
for i in range(tag_count):
    if exp_dict[i]!=0:
        out_dict[i]=out_dict[i]/exp_dict[i]*100
        
    else:
        out_dict[i]=0
def get_key(val):
    for key, value in tag_dict.items():
        if val == value:
            return key
for i in range(len(out_dict)):
    print(f'accuracy for {get_key(i)} tag is {out_dict[i]}' )
            

accuracy for <start> tag is 100.0
accuracy for PRP tag is 99.67948717948718
accuracy for VBP tag is 59.150326797385624
accuracy for VBN tag is 74.32835820895522
accuracy for CD tag is 99.64539007092199
accuracy for NNS tag is 92.56880733944955
accuracy for VBG tag is 93.63636363636364
accuracy for JJ tag is 89.61578400830737
accuracy for IN tag is 95.29072561268622
accuracy for VBD tag is 80.89171974522293
accuracy for NNP tag is 86.4406779661017
accuracy for <end>
 tag is 100.0
accuracy for DT tag is 99.23484402589759
accuracy for NN tag is 62.83574380165289
accuracy for VBZ tag is 92.87598944591029
accuracy for RB tag is 96.17224880382776
accuracy for JJR tag is 71.29629629629629
accuracy for TO tag is 100.0
accuracy for VB tag is 75.60975609756098
accuracy for CC tag is 100.0
accuracy for PRP$ tag is 93.6842105263158
accuracy for MD tag is 98.96907216494846
accuracy for NNPS tag is 56.25
accuracy for RBS tag is 0.0
accuracy for : tag is 96.66666666666667
accuracy for WDT tag is 100.